In [7]:
import torch 
import torch.nn as nn
import numpy as np
import scipy.io as sp
from functools import partial
from pyDOE import lhs
import time
import warnings
import scipy.io

N_train = 5000

 
# Load Data
data = scipy.io.loadmat('../Data/cylinder_nektar_wake.mat')
        
U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

######################################################################
######################## Noiseles Data ###############################
######################################################################
# Training Data    
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

x_train_pt = torch.from_numpy(x_train)
x_train_pt.requires_grad = True
y_train_pt = torch.from_numpy(y_train)
y_train_pt.requires_grad = True
t_train_pt = torch.from_numpy(t_train)
t_train_pt.requires_grad = True
u_train_pt = torch.from_numpy(u_train)
v_train_pt = torch.from_numpy(v_train)
 
lambda_1 = torch.nn.Parameter(torch.zeros(1, requires_grad=True))
lambda_2 = torch.nn.Parameter(torch.zeros(1, requires_grad=True))
 


In [15]:
input_tensor = torch.stack((x_train_pt, y_train_pt, t_train_pt), dim=1)
input_tensor.view(-1, 3).shape

torch.Size([5000, 2, 1])